In [1]:
import warnings
import os
warnings.filterwarnings("ignore")

os.makedirs('./data',exist_ok=True)
os.makedirs('./utils',exist_ok=True)

#download custom processing scripts if not already downloaded
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/kinematics.py
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/circstat.py
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/processing.py
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/skeleton.py

#download example data or upload your own json annotations

#--------------------------------------------------------------------------------------
# Download example raw data from figshare or specify path to your own json annotations |
#--------------------------------------------------------------------------------------

#!wget -P . https://figshare.com/ndownloader/articles/25316500/versions/1
# #unzip data into ./data folder and remove zip file
# !unzip ./1 -d ./data
# !rm ./1

In [2]:
#uncomment install if running on google colab
#%pip install scikit-video

import sys, os, cv2, glob, json, gc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import json
import numpy as np
import pandas as pd
import itertools
from itertools import chain
from moviepy.editor import VideoFileClip
import skvideo.io
from tqdm import tqdm
import circstat as CS
import scipy as sc
import math

from processing import *
from kinematics import *
from skeleton import *

In [ ]:
# format files as pkl with openpose standard and bodypart labels

from tqdm import tqdm

OVERWRITE = True
USE_CENTER_INSTANCE = False
USE_BEST_INSTANCE = True

dataset = 'gma_score_prediction'
json_path = f'./data/Infant Pose Data/{dataset}/annotations'
json_files = os.listdir(json_path)
directory = f'./data'

save_path = f'./pose_estimates/{dataset}_norm'

if not os.path.exists(save_path):
    os.makedirs(save_path)

kp_mapping = {0:'Nose', 1:'Neck', 2:'RShoulder', 3:'RElbow', 4:'RWrist', 5:'LShoulder', 6:'LElbow',
              7:'LWrist', 8:'RHip', 9:'RKnee', 10:'RAnkle', 11:'LHip',
              12:'LKnee', 13:'LAnkle', 14:'REye', 15:'LEye', 16:'REar', 17:'LEar'}

# Define the DataFrame columns as specified
columns = ['video_number', 'video', 'bp', 'frame', 'x', 'y', 'c','fps', 'pixel_x', 'pixel_y', 'time', 'part_idx']
data = []  # This will hold the data to be loaded into the DataFrame

vid_info = pd.read_csv(f'./data/{dataset}_video_info.csv')

for file_number, file in enumerate(tqdm(json_files)):
    # Construct the full file path
    file_path = os.path.join(json_path, file)
    fname = file.split('.')[0]
    interim = []

    if not OVERWRITE and os.path.exists(f'{save_path}/{fname}.pkl'):
        continue

    # Open and load the JSON data
    with open(file_path, 'r') as f:
        frames = json.load(f)
        info = vid_info[vid_info['video'] == fname]
        fps = vid_info['fps'].values[0]

        pixel_x = vid_info['width'].values[0]
        pixel_y = vid_info['height'].values[0]
        
        center_x = pixel_x / 2
        center_y = pixel_y / 2
        
        # Iterate through each frame in the JSON file
        for frame in frames:
            frame_id = frame['frame_id']
            if 'instances' in frame and len(frame['instances']) > 0:

                if USE_CENTER_INSTANCE:
                    instance_id = get_center_instance(frame['instances'], center_x, center_y)
                elif USE_BEST_INSTANCE:
                    instance_id = get_best_instance(frame['instances'])
                else:
                    instance_id = 0

                keypoints = frame['instances'][instance_id]['keypoints']
                confidence = frame['instances'][instance_id]['keypoint_scores']
                keypoints, confidence = convert_coco_to_openpose(keypoints, confidence)

                # Iterate through each keypoint
                for part_idx, (x, y) in enumerate(keypoints):

                    bp = kp_mapping[part_idx]
                    fps = fps
                    time = frame_id / fps
                    c = confidence[part_idx]

                    row = [file_number, fname, bp, frame_id, x, y, c, fps, pixel_x, pixel_y, time, part_idx]
                    interim.append(row)

    interim_df = pd.DataFrame(interim, columns=columns)
    interim_df.to_pickle(f'{save_path}/{fname}.pkl')

    del interim_df


  0%|          | 0/1057 [00:00<?, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
# Ensure the save_path directory exists
save_path = f'./pose_estimates/{dataset}_norm'

if os.path.exists(f'{save_path}/pose_estimates_{dataset}.csv'):
    os.remove(f'{save_path}/pose_estimates_{dataset}.csv')
    print('Removed existing CSV file')

for pklfile in tqdm(os.listdir(save_path)):
    if not pklfile.endswith('.pkl'):
        continue
    else:
        interim_df = pd.read_pickle(f'{save_path}/{pklfile}')
        interim_df.to_csv(f'{save_path}/pose_estimates_{dataset}.csv', mode='a', header=False, index=False)

    del interim_df

Removed existing CSV file


100%|██████████| 33/33 [00:22<00:00,  1.47it/s]


In [20]:
csv_path = f'{save_path}/pose_estimates_{dataset}.csv'
output_csv_path = f'{save_path}/pose_estimates_{dataset}_b.csv'
chunksize = 1000  # Number of rows per chunk

# Define the new headers
new_headers = ['video_number', 'video', 'bp', 'frame', 'x', 'y', 'c', 'fps', 'pixel_x', 'pixel_y', 'time', 'part_idx']

# Read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_path, chunksize=chunksize)

# Process the first chunk
first_chunk = next(chunk_iterator)
first_chunk.columns = new_headers
first_chunk.to_csv(output_csv_path, index=False)

# Process the rest of the chunks and append them to the new CSV file without headers
for chunk in chunk_iterator:
    chunk.columns = new_headers
    chunk.to_csv(output_csv_path, mode='a', index=False, header=False)

# rename the csv file
os.rename(csv_path, f'{save_path}/pose_estimates_{dataset}_x.csv')
os.rename(output_csv_path, csv_path)


In [21]:
def process_dataframe(df, pose_estimate_path):
    if df.empty:
        print("DataFrame is empty, skipping processing.")
        return

    print(f"Processing DataFrame for video_number: {df['video_number'].iloc[0]}")
    try:
        if dataset == 'Youtube':
            session = df['video'].unique()[0].split('_')[1][0]
            infant = df['video'].unique()[0].split('_')[1][3:]
        elif dataset == 'Clinical':
            # split based on naming convention
            session = df['video'].unique()[0].split('_')[1][1]
            infant = df['video'].unique()[0].split('_')[0][-1]

        age = 'month'

        print(f'infant: {infant}, session: {session}')

    except:
        f'could not process video {df["video"].unique()[0]}'
        return

    median_window = 1
    mean_window = 1
    delta_window = 0.25  # Smoothing applied to delta_x, velocity, acceleration

    df['x'] = pd.to_numeric(df['x'])
    df['y'] = pd.to_numeric(df['y'])

    # Interpolate
    df = df.groupby(['video', 'bp']).apply(interpolate_df).reset_index(drop=True)

    # Median and mean filter
    median_window = 0.5
    mean_window = 0.5
    df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'y', median_window, mean_window)).reset_index(drop=True)
    df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'x', median_window, mean_window)).reset_index(drop=True)

    try:
        # Rotate and normalise by reference
        xdf = normalise_skeletons(df)
        xdf = get_dynamics_xy(xdf, delta_window)
        xdf.to_pickle(f'{pose_estimate_path}/xdf/{infant}_{session}_processed_pose_estimates_coords_norm.pkl')

        adf = get_joint_angles(df)
        adf = get_dynamics_angle(adf, delta_window)
        adf.to_pickle(f'{pose_estimate_path}/adf/{infant}_{session}_processed_pose_estimates_coords_norm.pkl')

    except KeyError as e:
        print(f'Error processing video for {infant}: {e}')


In [22]:
# Smooth detections and compute features

pose_estimate_path = f'./pose_estimates/{dataset}_norm'
csv_path = f'{pose_estimate_path}/pose_estimates_{dataset}.csv'
save_path = f'{pose_estimate_path}/pose_estimates_{dataset}_processed.csv'

# Create necessary directories
os.makedirs(pose_estimate_path, exist_ok=True)
os.makedirs(f'{pose_estimate_path}/xdf', exist_ok=True)
os.makedirs(f'{pose_estimate_path}/adf', exist_ok=True)

In [23]:
chunksize = 100000
buffer = pd.DataFrame()

# Read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_path, chunksize=chunksize)

for chunk in chunk_iterator:
  unique_videos = chunk['video_number'].unique()

  for video_number in unique_videos:
      video_chunk = chunk[chunk['video_number'] == video_number]

      if not buffer.empty:
        if buffer['video_number'].iloc[0] == video_number:

            buffer = pd.concat([buffer, video_chunk], ignore_index=True)
            if video_number not in chunk['video_number'].values:
                process_dataframe(buffer, pose_estimate_path)
                buffer = pd.DataFrame()

        else:
            process_dataframe(buffer, pose_estimate_path)
            buffer = video_chunk
      else:
          buffer = video_chunk

  chunk = chunk[~chunk['video_number'].isin(unique_videos)]

# # Process any remaining rows in the buffer
if not buffer.empty:
    print("Processing remaining rows in the buffer...")
    process_dataframe(buffer, pose_estimate_path)

Processing DataFrame for video_number: 0
Processing DataFrame for video_number: 1
Processing DataFrame for video_number: 2
Processing DataFrame for video_number: 3
Processing DataFrame for video_number: 4
infant: 7, session: 0
Processing DataFrame for video_number: 5
Processing DataFrame for video_number: 6
Processing DataFrame for video_number: 7
Processing DataFrame for video_number: 8
Processing DataFrame for video_number: 9
Processing DataFrame for video_number: 10
Processing DataFrame for video_number: 11
Processing DataFrame for video_number: 12
Processing DataFrame for video_number: 13
Processing DataFrame for video_number: 14
Processing DataFrame for video_number: 15
Processing DataFrame for video_number: 16
Processing DataFrame for video_number: 17
Processing DataFrame for video_number: 18
Processing DataFrame for video_number: 19
Processing DataFrame for video_number: 20
Processing DataFrame for video_number: 21
Processing DataFrame for video_number: 22
Processing DataFrame f

In [24]:
# Compute angular features

HEADER_WRITTEN = False

for file in tqdm(os.listdir(f'{pose_estimate_path}/adf')):
    if 'pkl' in file:

        adf = pd.read_pickle(os.path.join(f'{pose_estimate_path}/adf', file))

        # angle features
        feature_angle = adf.groupby(['bp','video']).apply(angle_features).reset_index(drop=True)
        feature_angle = pd.pivot_table(feature_angle, index='video', columns=['bp'])
        l0 = feature_angle.columns.get_level_values(1)
        l1 = feature_angle.columns.get_level_values(0)
        cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
        feature_angle.columns = cols
        feature_angle =feature_angle.reset_index()

        # - measure of symmetry (left-right cross correlation)
        corr_joint = adf.groupby(['video', 'part']).apply(lambda x:corr_lr(x,'angle')).reset_index()
        corr_joint['part'] = 'lrCorr_angle_'+corr_joint['part']
        corr_joint.columns = ['video', 'feature', 'Value']
        corr_joint = pd.pivot_table(corr_joint, index='video', columns=['feature'])
        l1 = corr_joint.columns.get_level_values(1)
        corr_joint.columns = l1
        corr_joint = corr_joint.reset_index()
        feature_angle = pd.merge(feature_angle,corr_joint, on='video', how='outer')

        if not HEADER_WRITTEN:
            feature_angle.to_csv(f'{pose_estimate_path}/features_angle.csv', header=True, index=False)
            HEADER_WRITTEN = True
        else:
            feature_angle.to_csv(f'{pose_estimate_path}/features_angle.csv', header=False, index=False, mode='a')

100%|██████████| 2/2 [00:00<00:00, 10.72it/s]


In [26]:
# Compute XY features

HEADER_WRITTEN = False

for file in tqdm(os.listdir(f'{pose_estimate_path}/xdf')):
    if 'pkl' in file:

        xdf = pd.read_pickle(os.path.join(f'{pose_estimate_path}/xdf', file))

        # xy features
        bps = ['LAnkle', 'RAnkle', 'LWrist', 'RWrist']
        feature_xy = xdf[np.isin(xdf.bp, bps)].groupby(['bp','video']).apply(xy_features).reset_index(drop=True)
        feature_xy = pd.pivot_table(feature_xy, index='video', columns=['bp'])
        l0 = feature_xy.columns.get_level_values(1)
        l1 = feature_xy.columns.get_level_values(0)
        cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
        feature_xy.columns = cols
        feature_xy = feature_xy.reset_index()
        # - measure of symmetry (left-right cross correlation)
        xdf['dist'] = np.sqrt(xdf['x']**2+xdf['y']**2)
        corr_joint = xdf.groupby(['video', 'part']).apply(lambda x:corr_lr(x,'dist')).reset_index()
        corr_joint['part'] = 'lrCorr_x_'+corr_joint['part']
        corr_joint.columns = ['video', 'feature', 'Value']
        corr_joint = pd.pivot_table(corr_joint, index='video', columns=['feature'])
        l1 = corr_joint.columns.get_level_values(1)
        corr_joint.columns = l1
        corr_joint = corr_joint.reset_index()
        feature_xy = pd.merge(feature_xy, corr_joint, on='video', how='outer')

        if not HEADER_WRITTEN:
            feature_xy.to_csv(f'{pose_estimate_path}/features_xy.csv', header=True, index=False, mode='a')
            HEADER_WRITTEN = True
        else:
            feature_xy.to_csv(f'{pose_estimate_path}/features_xy.csv', header=False, index=False, mode='a')

100%|██████████| 2/2 [00:00<00:00,  8.45it/s]


In [27]:
# Combine features
SAVE = True

features_xy = pd.read_csv(f'{pose_estimate_path}/features_xy.csv')
features_angle = pd.read_csv(f'{pose_estimate_path}/features_angle.csv')

features = pd.merge(features_xy, features_angle, on='video', how='inner')

In [28]:
# Split features based on video file naming convention

features['infant'] = features['video'].str.split('_').str.get(1).str[-3:]
features['session'] = features['video'].str.split('_').str.get(1).str[0]
features['age'] = 'month'

if SAVE:
    features.to_csv(f'{pose_estimate_path}/features.csv', header=True, index=False)

In [29]:
# Body parts and sides
body_parts = ["Knee", "Shoulder", "Hip", "Elbow", "Wrist", "Ankle"]
sides = ["L", "R"]

# Function to split the feature string into "feature", "part", and "side"
def split_feature(feature):
    # Split the feature into components
    parts = feature.split('_')
    feature_name = "_".join(parts[:-1])  # Default to everything before the last part
    part = parts[-1] if len(parts) > 1 else ""  # Default to the last part

    side = ""

    # Check if the last part has a side
    for body_part in body_parts:
        if body_part in part:  # Find body part
            idx = part.index(body_part)
            if idx > 0 and part[idx-1] in sides:  # Check for side prefix
                side = part[idx-1]
                part = part[idx:]  # Remove the side from the part
            break  # Exit loop after finding the body part

    # Adjust feature name to remove body part and side, if found
    if part in feature_name:
        feature_name = feature_name.replace(part, "").strip("_")
    if side in feature_name:
        feature_name = feature_name.replace(side, "").strip("_")

    return pd.Series([feature_name, part, side])

In [30]:
# Include data for the following, and restructure dataframe
# Customize for your specific dataset

id_vars = ['infant', 'age','session','video']
melted = pd.melt(features, id_vars=id_vars, var_name="feature", value_name="Value")

# Apply the function to the 'feature' column in your melted DataFrame
melted[['feature', 'part', 'side']] = melted['feature'].apply(split_feature)
melted = melted.dropna()
mean = melted.groupby(['infant', 'age', 'session', 'video', 'feature', 'part'])['Value'].mean().reset_index()
mean.to_csv(f'{pose_estimate_path}/{dataset}_features_mean_by_side.csv', header=True, index=False)

mean.head()

,infant,age,session,video,feature,part,Value
0,11,month,1,822487_11_3_1_gp1_1,IQR_acc_angle,Elbow,128.009737
1,11,month,1,822487_11_3_1_gp1_1,IQR_acc_angle,Hip,84.806122
2,11,month,1,822487_11_3_1_gp1_1,IQR_acc_angle,Knee,86.910725
3,11,month,1,822487_11_3_1_gp1_1,IQR_acc_angle,Shoulder,62.897430
4,11,month,1,822487_11_3_1_gp1_1,IQR_vel_angle,Elbow,32.885301
